In [251]:
%load_ext autoreload
%autoreload 2

../data/models/
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [267]:
import numpy as np
import pandas as pd
import torch
import re
from sklearn.metrics import balanced_accuracy_score, roc_auc_score,accuracy_score
from Constants import *
from Preprocessing import *
from Models import *
pd.set_option('display.max_rows', 200)



In [192]:
pd.read_csv('../data/digital_twin_ln_data.csv')

,id,cluster,1A1B,1A6,1B2A,1B3,2A2B,2A3,2B5A,34,...,1A,1B,2A,2B,3,4,5A,5B,6,RPLN
0,1,1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,5089,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5088,1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,5087,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5085,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,1033726,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1176,1033946,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1177,1035327,1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1178,1035635,1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [195]:
pd.read_csv('../data/digital_twin_ln_monograms.csv')

,id,L1A,L1B,L2A,L2B,L3,L4,L5A,L5B,L6,...,R1A,R1B,R2A,R2B,R3,R4,R5A,R5B,R6,RRPLN
0,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,10201,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
532,10202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
533,10203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
534,10204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [262]:
data = DTDataset(use_smote=False,smote_ids = Const.stratified_train_ids,ln_data_file = '../data/digital_twin_ln_monograms.csv')
data.processed_df#.shape, len(data.processed_df.index.unique())

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


,hpv,age,packs_per_year,smoking_status,gender,Aspiration rate Pre-therapy,total_dose,dose_fraction,bilateral,cc_none,...,4_ipsi,4_contra,5A_ipsi,5A_contra,5B_ipsi,5B_contra,6_ipsi,6_contra,RPLN_ipsi,RPLN_contra
id,,,,,,,,,,,,,,,,,,,,,
3,1,55.969444,0.0,0.0,1,0,66.00,2.200000,False,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,20.950000,38.0,1.0,1,0,72.00,1.800000,False,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,69.930556,35.0,1.0,0,1,70.00,2.121212,True,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1,72.319444,0.0,1.0,1,0,70.00,2.121212,False,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,59.730556,0.0,0.0,1,0,66.00,2.200000,False,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10201,1,49.566667,30.0,1.0,1,0,70.00,2.121212,False,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10202,0,48.705556,30.0,1.0,1,0,72.00,1.714286,False,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10203,1,77.116667,0.0,0.0,1,0,70.00,2.333333,False,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [354]:
data.processed_df.loc[5091]

hpv                                         0
age                                 74.302778
packs_per_year                           40.0
smoking_status                            0.5
gender                                      1
Aspiration rate Pre-therapy                 0
total_dose                               66.0
dose_fraction                             2.2
bilateral                               False
cc_none                                     0
cc_platinum                                 1
cc_cetuximab                                0
cc_others                                   0
no_dose_adjustment                          1
dose_modified                               0
dose_delayed                                0
dose_cancelled                              0
dose_delayed_&_modified                     0
regiment_modification                       0
unknown                                     0
T-category_1                                0
T-category_2                      

In [4]:
model_dir = Const.data_dir + '/models/'

tuned_transition_models = [
    'final_transition1_model_state1_input63_dims500,500_dropout0.25,0.5.pt',
    'final_transition2_model_state2_input85_dims100_dropout0.25,0.pt',
    'final_outcome_model_state1_input83_dims1000_dropout0,0.pt'
]
tuned_transition_models = [model_dir + f for f in tuned_transition_models]
Const.tuned_transition_models = tuned_transition_models
Const.tuned_decision_model = model_dir +  'final_decision_model_statedecisions_input132_dims100,100_dropout0.1,0.7.pt'

In [241]:
def get_dt_ids():
    df = load_digital_twin()
    return df.id.values

def get_tt_split(ids=None,use_default_split=True,use_bagging_split=False,resample_training=False):
        if ids is None:
            ids = get_dt_ids()
        #pre-made, stratified by decision and outcome 72:28
        if use_default_split:
            train_ids = Const.stratified_train_ids[:]
            test_ids = Const.stratified_test_ids[:]
        elif use_bagging_split:
            train_ids = np.random.choice(ids,len(ids),replace=True)
            test_ids = [i for i in ids if i not in train_ids]
        else:
            test_ids = ids[0: int(len(ids)*(1-split))]
            train_ids = [i for i in ids if i not in test_ids]

        if resample_training:
            train_ids = np.random.choice(train_ids,len(train_ids),replace=True)
            test_ids = [i for i in ids if i not in train_ids]
        return train_ids,test_ids
    
def df_to_torch(df,ttype  = torch.FloatTensor):
    values = df.values.astype(float)
    values = torch.from_numpy(values)
    return values.type(ttype)

In [242]:
def nllloss(ytrue,ypred):
    #nll loss with argmax added in
    loss = torch.nn.NLLLoss()
    return loss(ypred,ytrue.argmax(axis=1))

def state_loss(ytrue,ypred,weights=[1,1,1,1]):
    pd_loss = nllloss(ytrue[0],ypred[0])*weights[0]
    nd_loss = nllloss(ytrue[1],ypred[1])*weights[1]
    mod_loss = nllloss(ytrue[2],ypred[2])*weights[2]
    loss = pd_loss + nd_loss + mod_loss
    dlt_true = ytrue[3]
    dlt_pred = ypred[3]
    ndlt = dlt_true.shape[1]
#     nloss = torch.nn.NLLLoss()
    bce = torch.nn.BCELoss()
    for i in range(ndlt):
        dlt_loss = bce(dlt_pred[:,i].view(-1),dlt_true[:,i].view(-1))
        loss += dlt_loss*weights[3]/ndlt
    return loss

def outcome_loss(ytrue,ypred,weights=[1,1,1]):
    loss = 0
    nloss = torch.nn.BCELoss()
    for i in range(len(weights)):
        iloss = nloss(ypred[:,i],ytrue[i])*weights[i]
        loss += iloss
    return loss

def mc_metrics(yt,yp,numpy=False,is_dlt=False):
    if not numpy:
        yt = yt .cpu().detach().numpy()
        yp = yp.cpu().detach().numpy()
    #dlt prediction (binary)
    if is_dlt:
        acc = accuracy_score(yt,yp>.5)
        if yt.sum() > 1:
            auc = roc_auc_score(yt,yp)
        else:
            auc=-1
        error = np.mean((yt-yp)**2)
        return {'accuracy': acc, 'mse': error, 'auc': auc}
    #this is a catch for when I se the dlt prediction format (encoded integer ordinal, predict as a categorical and take the argmax)
    elif yt.ndim > 1:
        try:
            bacc = balanced_accuracy_score(yt.argmax(axis=1),yp.argmax(axis=1))
        except:
            bacc = -1
        try:
            roc_micro = roc_auc_score(yt,yp,average='micro')
        except:
            roc_micro=-1
        try:
            roc_macro = roc_auc_score(yt,yp,average='macro')
        except:
            roc_macro = -1
        return {'accuracy': bacc, 'roc_micro': roc_micro,'roc_macro': roc_macro}
    #outcomes (binary)
    else:
        if yp.ndim > 1:
            yp = yp.argmax(axis=1)
        try:
            bacc = accuracy_score(yt,yp)
        except:
            bacc = -1
        try:
            roc = roc_auc_score(yt,yp)
        except:
            roc = -1
        error = np.mean((yt-yp)**2)
        return {'accuracy': bacc, 'mse': error, 'auc': roc}

def state_metrics(ytrue,ypred,numpy=False):
    pd_metrics = mc_metrics(ytrue[0],ypred[0],numpy=numpy)
    nd_metrics = mc_metrics(ytrue[1],ypred[1],numpy=numpy)
    mod_metrics = mc_metrics(ytrue[1],ypred[1],numpy=numpy)
    
    dlt_metrics = []
    dlt_true = ytrue[3]
    dlt_pred = ypred[3]
    ndlt = dlt_true.shape[1]
    nloss = torch.nn.NLLLoss()
    for i in range(ndlt):
        dm = mc_metrics(dlt_true[:,i],dlt_pred[:,i].view(-1),is_dlt=True)
        dlt_metrics.append(dm)
    dlt_acc =[d['accuracy'] for d in dlt_metrics]
    dlt_error = [d['mse'] for d in dlt_metrics]
    dlt_auc = [d['auc'] for d in dlt_metrics]
    return {'pd': pd_metrics,'nd': nd_metrics,'mod': mod_metrics,'dlts': {'accuracy': dlt_acc,'accuracy_mean': np.mean(dlt_acc),'auc': dlt_auc,'auc_mean': np.mean(dlt_auc)}}
    
def outcome_metrics(ytrue,ypred,numpy=False):
    res = {}
    for i, outcome in enumerate(Const.outcomes):
        metrics = mc_metrics(ytrue[i],ypred[:,i])
        res[outcome] = metrics
    return res


In [44]:
from sklearn.ensemble import RandomForestClassifier

def train_state_rf(model_args={}):
    ids = get_dt_ids()
    
    dataset = DTDataset()

    train_ids = ids[0:int(len(ids)*.7)]
    test_ids = ids[int(len(ids)*.7):]
    
    #most things are multiclass, dlts are several ordinal and outcomes are multiple binary
    xtrain1 = dataset.get_state('baseline',ids=train_ids)
    xtest1 = dataset.get_state('baseline',ids=test_ids)
    
    xtrain2 = dataset.get_input_state(step=2,ids=train_ids)
    xtest2 = dataset.get_input_state(step=2,ids=test_ids)
    
    xtrain3 = dataset.get_input_state(step=3,ids=train_ids)
    xtest3 = dataset.get_input_state(step=3,ids=test_ids)
    
    [pd1_train,nd1_train, mod_train,dlts1_train] = dataset.get_intermediate_outcomes(ids=train_ids)
    [pd2_train,nd2_train, cc_train,dlts2_train] = dataset.get_intermediate_outcomes(step=2,ids=train_ids)
    [pd1_test,nd1_test, mod_test,dlts1_test] = dataset.get_intermediate_outcomes(ids=test_ids)
    [pd2_test,nd2_test, cc_test,dlts2_test] = dataset.get_intermediate_outcomes(step=2,ids=test_ids)
    outcomes_train = dataset.get_state('outcomes',ids=train_ids)
    outcomes_test = dataset.get_state('outcomes',ids=test_ids)
    

    def train_multiclass_rf(xtrain,xtest,ytrain,ytest):
        model = RandomForestClassifier(class_weight='balanced',**model_args).fit(xtrain,ytrain)
        ypred = model.predict(xtest)
        metrics = mc_metrics(ytest.values,ypred,numpy=True)
        return model, metrics
    
    all_metrics = {}
    pd1_model, all_metrics['pd1'] = train_multiclass_rf(xtrain1,xtest1,pd1_train,pd1_test)
    nd1_model, all_metrics['nd1']  = train_multiclass_rf(xtrain1,xtest1,nd1_train,nd1_test)
    mod_model, all_metrics['mod']  = train_multiclass_rf(xtrain1,xtest1,mod_train,mod_test)
    
    return all_metrics

train_state_rf({'max_depth': 5,'n_estimators': 100})

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  


Index(['hpv', 'age', 'packs_per_year', 'smoking_status', 'gender',
       'Aspiration rate Pre-therapy', 'total_dose', 'dose_fraction', '1A1B',
       '1A6',
       ...
       'SD Primary 2', 'CR Nodal 2', 'PR Nodal 2', 'SD Nodal 2',
       'Decision 1 (Induction Chemo) Y/N', 'Decision 2 (CC / RT alone)',
       'Decision 3 Neck Dissection (Y/N)', 'Overall Survival (4 Years)', 'FT',
       'Aspiration rate Post-therapy'],
      dtype='object', length=111)


{'pd1': {'accuracy': 0.36326291079812206,
  'roc_micro': 0.5126910126910126,
  'roc_macro': 0.5150862068965517},
 'nd1': {'accuracy': 0.38838037121789987,
  'roc_micro': 0.5717430717430717,
  'roc_macro': 0.5260869565217391},
 'mod': {'accuracy': 0.16666666666666666,
  'roc_micro': 0.515527950310559,
  'roc_macro': -1}}

In [243]:
 DTDataset().get_state('baseline').T

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


id,3,5,6,7,8,9,10,11,13,14,...,10196,10197,10198,10199,10200,10201,10202,10203,10204,10205
1A_contra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1A_ipsi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1B_contra,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1B_ipsi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2A_contra,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2A_ipsi,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
2B_contra,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2B_ipsi,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
3_contra,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3_ipsi,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0


In [355]:
def train_state(model=None,
                model_args={},
                state=1,
                split=.7,
                lr=.0001,
                epochs=1000,
                patience=10,
                use_attention=True,
                weights=[1,1,1,10],
                save_path='../data/models/',
                use_default_split=True,
                use_bagging_split=False,
                resample_training=False,#use bootstraping on training data after splitting
                n_validation_trainsteps=2,
                verbose=True,
                use_smote=True,
                file_suffix=''):
    
    ids = get_dt_ids()
    
    train_ids, test_ids = get_tt_split(use_default_split=use_default_split,use_bagging_split=use_bagging_split,resample_training=resample_training)
    
    if use_smote:
        dataset = DTDataset(use_smote=True,smote_ids = train_ids)
        train_ids = [i for i in dataset.processed_df.index.values if i not in test_ids]
    else:
        dataset = DTDataset()

    xtrain = dataset.get_input_state(step=state,ids=train_ids)
    xtest = dataset.get_input_state(step=state,ids=test_ids)
    ytrain = dataset.get_intermediate_outcomes(step=state,ids=train_ids)
    ytest = dataset.get_intermediate_outcomes(step=state,ids=test_ids)
    

    if not use_attention:
        model_args = {k:v for k,v in model_args.items() if 'attention' not in k and 'embed_size' not in k}
    if state < 3:
        if model is None:
            if use_attention:
                model = OutcomeAttentionSimulator(xtrain.shape[1],state=state,**model_args)
            else:
                model = OutcomeSimulator(xtrain.shape[1],state=state,**model_args)
        lfunc = state_loss
    else:
        if model is None:
            if use_attention:
                model = EndpointAttentionSimulator(xtrain.shape[1],**model_args)
            else:
                model = EndpointSimulator(xtrain.shape[1],**model_args)
        weights = weights[:3]
        lfunc = outcome_loss
        
    hashcode = str(hash(','.join([str(i) for i in train_ids])))
    save_file = save_path + 'model_' + model.identifier + '_split' + str(split) + '_resample' + str(resample_training) +  '_hash' + hashcode + file_suffix + '.tar'
    xtrain = df_to_torch(xtrain)
    xtest = df_to_torch(xtest)
    ytrain = [df_to_torch(t) for t in ytrain]
    ytest= [df_to_torch(t) for t in ytest]
    
    model.fit_normalizer(xtrain)
#     normalize = lambda x: (x - xtrain.mean(axis=0)+.01)/(xtrain.std(axis=0)+.01)
#     unnormalize = lambda x: (x * (xtrain.std(axis=0) +.01)) + xtrain.mean(axis=0) - .01
    
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    best_val_loss = 1000000000000000000000000000
    best_loss_metrics = {}
    last_epoch = False
    for epoch in range(epochs):
        
        model.train(True)
        optimizer.zero_grad()
        
        xtrain_sample = xtrain#[torch.randint(len(xtrain),(len(xtrain),) )]
        ypred = model(xtrain_sample)
        loss = lfunc(ytrain,ypred,weights=weights)

        loss.backward()
        optimizer.step()
        if verbose:
            print('epoch',epoch,'train loss',loss.item())
        
        model.eval()
        yval = model(xtest)
        val_loss = lfunc(ytest,yval,weights=weights)
        if state < 3:
            val_metrics = state_metrics(ytest,yval)
        else:
            val_metrics = outcome_metrics(ytest,yval)
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            best_loss_metrics = val_metrics
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
        if verbose:
            print('val loss',val_loss.item())
            print('______________')
        if steps_since_improvement > patience:
            break
    print('best loss',best_val_loss,best_loss_metrics)
    model.load_state_dict(torch.load(save_file))
    
    #train one step on validation data
    for i in range(n_validation_trainsteps):
        model.train()
        yval = model(xtest)
        val_loss = lfunc(ytest,yval,weights=weights)
        val_loss.backward()
        optimizer.step()
        torch.save(model.state_dict(),save_file)
    
    model.eval()
    return model,  best_val_loss, best_loss_metrics

t1_args = {'hidden_layers': [1000], 'dropout': 0.9, 'input_dropout': 0.5}
model1,_,_ = train_state(model_args=t1_args,use_attention=False,use_smote=False)
model1

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


epoch 0 train loss 13.470033645629883
val loss 11.2970552444458
______________
epoch 1 train loss 13.214168548583984
val loss 11.133837699890137
______________
epoch 2 train loss 13.090332984924316
val loss 10.974069595336914
______________
epoch 3 train loss 12.935943603515625
val loss 10.81753158569336
______________
epoch 4 train loss 12.650671005249023
val loss 10.664196014404297
______________
epoch 5 train loss 12.583200454711914
val loss 10.513541221618652
______________
epoch 6 train loss 12.080845832824707
val loss 10.365581512451172
______________
epoch 7 train loss 11.798055648803711
val loss 10.220637321472168
______________
epoch 8 train loss 11.700197219848633
val loss 10.078390121459961
______________
epoch 9 train loss 11.420167922973633
val loss 9.938918113708496
______________
epoch 10 train loss 11.699029922485352
val loss 9.802186965942383
______________
epoch 11 train loss 11.395209312438965
val loss 9.66819953918457
______________
epoch 12 train loss 11.0049819946

epoch 110 train loss 4.557486057281494
val loss 4.36065673828125
______________
epoch 111 train loss 4.5827531814575195
val loss 4.341801643371582
______________
epoch 112 train loss 4.554055213928223
val loss 4.323163032531738
______________
epoch 113 train loss 4.7328782081604
val loss 4.304826259613037
______________
epoch 114 train loss 4.498247146606445
val loss 4.286726474761963
______________
epoch 115 train loss 4.532895088195801
val loss 4.268918991088867
______________
epoch 116 train loss 4.44027853012085
val loss 4.25132942199707
______________
epoch 117 train loss 4.307986736297607
val loss 4.233956336975098
______________
epoch 118 train loss 4.483946323394775
val loss 4.2168474197387695
______________
epoch 119 train loss 4.356116771697998
val loss 4.1999053955078125
______________
epoch 120 train loss 4.480731964111328
val loss 4.183228015899658
______________
epoch 121 train loss 4.489027976989746
val loss 4.166811943054199
______________
epoch 122 train loss 4.2840666

val loss 3.209648847579956
______________
epoch 213 train loss 3.391664743423462
val loss 3.202810049057007
______________
epoch 214 train loss 3.4428515434265137
val loss 3.1960768699645996
______________
epoch 215 train loss 3.467329740524292
val loss 3.18940806388855
______________
epoch 216 train loss 3.4496898651123047
val loss 3.182798385620117
______________
epoch 217 train loss 3.4641168117523193
val loss 3.1762795448303223
______________
epoch 218 train loss 3.487886428833008
val loss 3.1698527336120605
______________
epoch 219 train loss 3.4671506881713867
val loss 3.163487195968628
______________
epoch 220 train loss 3.4526169300079346
val loss 3.157176971435547
______________
epoch 221 train loss 3.3251447677612305
val loss 3.150928020477295
______________
epoch 222 train loss 3.316899061203003
val loss 3.144719123840332
______________
epoch 223 train loss 3.5185577869415283
val loss 3.1385908126831055
______________
epoch 224 train loss 3.4671056270599365
val loss 3.132474

val loss 2.706130027770996
______________
epoch 320 train loss 3.0929017066955566
val loss 2.703071117401123
______________
epoch 321 train loss 2.8910391330718994
val loss 2.6999683380126953
______________
epoch 322 train loss 3.020517110824585
val loss 2.6969704627990723
______________
epoch 323 train loss 2.992154836654663
val loss 2.693970203399658
______________
epoch 324 train loss 3.025301456451416
val loss 2.691044807434082
______________
epoch 325 train loss 2.984372615814209
val loss 2.6880674362182617
______________
epoch 326 train loss 3.111245632171631
val loss 2.6850943565368652
______________
epoch 327 train loss 3.024503469467163
val loss 2.6820874214172363
______________
epoch 328 train loss 3.0138776302337646
val loss 2.6790502071380615
______________
epoch 329 train loss 3.027296543121338
val loss 2.6760475635528564
______________
epoch 330 train loss 3.053953170776367
val loss 2.673074722290039
______________
epoch 331 train loss 3.0948398113250732
val loss 2.670160

...

val loss 2.197364091873169
______________
epoch 639 train loss 2.423638105392456
val loss 2.196611166000366
______________
epoch 640 train loss 2.3484134674072266
val loss 2.1958463191986084
______________
epoch 641 train loss 2.3313229084014893
val loss 2.1950254440307617
______________
epoch 642 train loss 2.4148924350738525
val loss 2.194106101989746
______________
epoch 643 train loss 2.3777124881744385
val loss 2.193140983581543
______________
epoch 644 train loss 2.4476308822631836
val loss 2.192180871963501
______________
epoch 645 train loss 2.4658005237579346
val loss 2.191211223602295
______________
epoch 646 train loss 2.4195096492767334
val loss 2.190227746963501
______________
epoch 647 train loss 2.3667447566986084
val loss 2.1891822814941406
______________
epoch 648 train loss 2.3907663822174072
val loss 2.188216209411621
______________
epoch 649 train loss 2.3348028659820557
val loss 2.187274694442749
______________
epoch 650 train loss 2.4062490463256836
val loss 2.186

val loss 2.1365528106689453
______________
epoch 746 train loss 2.421518564224243
val loss 2.1360418796539307
______________
epoch 747 train loss 2.288177251815796
val loss 2.1354470252990723
______________
epoch 748 train loss 2.4203829765319824
val loss 2.1347291469573975
______________
epoch 749 train loss 2.3228461742401123
val loss 2.134099006652832
______________
epoch 750 train loss 2.277045965194702
val loss 2.1333999633789062
______________
epoch 751 train loss 2.3447539806365967
val loss 2.1326446533203125
______________
epoch 752 train loss 2.156909465789795
val loss 2.1318273544311523
______________
epoch 753 train loss 2.343763589859009
val loss 2.1309995651245117
______________
epoch 754 train loss 2.264204263687134
val loss 2.130128860473633
______________
epoch 755 train loss 2.287170171737671
val loss 2.1293118000030518
______________
epoch 756 train loss 2.2493419647216797
val loss 2.1284711360931396
______________
epoch 757 train loss 2.285388469696045
val loss 2.127

val loss 2.0934395790100098
______________
epoch 852 train loss 2.406705856323242
val loss 2.0931806564331055
______________
epoch 853 train loss 2.3105149269104004
val loss 2.0930094718933105
______________
epoch 854 train loss 2.2468016147613525
val loss 2.0928287506103516
______________
epoch 855 train loss 2.276496410369873
val loss 2.092653274536133
______________
epoch 856 train loss 2.4028759002685547
val loss 2.0924646854400635
______________
epoch 857 train loss 2.2224371433258057
val loss 2.092186450958252
______________
epoch 858 train loss 2.2755298614501953
val loss 2.0919830799102783
______________
epoch 859 train loss 2.2336840629577637
val loss 2.0917136669158936
______________
epoch 860 train loss 2.2495858669281006
val loss 2.0914645195007324
______________
epoch 861 train loss 2.121397018432617
val loss 2.091163396835327
______________
epoch 862 train loss 2.323371410369873
val loss 2.0910232067108154
______________
epoch 863 train loss 2.240656614303589
val loss 2.0

val loss 2.064159631729126
______________
epoch 959 train loss 2.1094861030578613
val loss 2.0639376640319824
______________
epoch 960 train loss 2.162257671356201
val loss 2.063725471496582
______________
epoch 961 train loss 2.254152297973633
val loss 2.0635952949523926
______________
epoch 962 train loss 2.2359261512756348
val loss 2.06352162361145
______________
epoch 963 train loss 2.095539093017578
val loss 2.0634758472442627
______________
epoch 964 train loss 2.111147880554199
val loss 2.063382863998413
______________
epoch 965 train loss 2.2358953952789307
val loss 2.0633201599121094
______________
epoch 966 train loss 2.282724380493164
val loss 2.0634093284606934
______________
epoch 967 train loss 2.0360641479492188
val loss 2.063382863998413
______________
epoch 968 train loss 2.182522773742676
val loss 2.0633492469787598
______________
epoch 969 train loss 2.1436967849731445
val loss 2.063270330429077
______________
epoch 970 train loss 2.2676637172698975
val loss 2.063189

OutcomeSimulator(
  (input_dropout): Dropout(p=0.5, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=50, out_features=1000, bias=True)
    (1): ReLU()
  )
  (batchnorm): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.9, inplace=False)
  (sigmoid): Sigmoid()
  (softmax): LogSoftmax(dim=1)
  (disease_layer): Linear(in_features=1000, out_features=3, bias=True)
  (nodal_disease_layer): Linear(in_features=1000, out_features=3, bias=True)
  (dlt_layers): ModuleList(
    (0): Linear(in_features=1000, out_features=1, bias=True)
    (1): Linear(in_features=1000, out_features=1, bias=True)
    (2): Linear(in_features=1000, out_features=1, bias=True)
    (3): Linear(in_features=1000, out_features=1, bias=True)
    (4): Linear(in_features=1000, out_features=1, bias=True)
    (5): Linear(in_features=1000, out_features=1, bias=True)
    (6): Linear(in_features=1000, out_features=1, bias=True)
    (7): Linear(in_features=10

In [356]:
t2_args = {'hidden_layers': [500, 500], 'dropout': 0.9, 'input_dropout': 0.5}
model2,_,_ = train_state(model_args=t2_args,state=2,use_attention=False,use_smote=False)
model2

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


epoch 0 train loss 10.844374656677246
val loss 10.376598358154297
______________
epoch 1 train loss 10.724784851074219
val loss 10.22596263885498
______________
epoch 2 train loss 10.525107383728027
val loss 10.082180976867676
______________
epoch 3 train loss 10.319523811340332
val loss 9.94363021850586
______________
epoch 4 train loss 10.250266075134277
val loss 9.809475898742676
______________
epoch 5 train loss 9.961845397949219
val loss 9.679276466369629
______________
epoch 6 train loss 9.825467109680176
val loss 9.552268028259277
______________
epoch 7 train loss 9.61941146850586


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains clas

val loss 9.42834758758545
______________
epoch 8 train loss 9.569151878356934
val loss 9.306841850280762
______________
epoch 9 train loss 9.315780639648438
val loss 9.187287330627441
______________
epoch 10 train loss 9.312115669250488
val loss 9.069486618041992
______________
epoch 11 train loss 9.063375473022461
val loss 8.95294189453125
______________
epoch 12 train loss 8.973291397094727
val loss 8.83779525756836
______________
epoch 13 train loss 8.85146713256836
val loss 8.723657608032227
______________
epoch 14 train loss 8.746414184570312
val loss 8.61051082611084
______________
epoch 15 train loss 8.617608070373535
val loss 8.4981050491333
______________
epoch 16 train loss 8.466524124145508
val loss 8.386335372924805
______________
epoch 17 train loss 8.400378227233887
val loss 8.275148391723633
______________
epoch 18 train loss 8.181365013122559
val loss 8.164520263671875
______________
epoch 19 train loss 8.17809009552002
val loss 8.054410934448242
______________
epoch 20

val loss 3.80308198928833
______________
epoch 111 train loss 4.3055572509765625
val loss 3.7962276935577393
______________
epoch 112 train loss 4.115095138549805
val loss 3.789747953414917
______________
epoch 113 train loss 4.208077907562256
val loss 3.783501148223877
______________
epoch 114 train loss 4.148627758026123
val loss 3.7774012088775635
______________
epoch 115 train loss 4.031924724578857
val loss 3.771388530731201
______________
epoch 116 train loss 4.197422504425049
val loss 3.7655134201049805
______________
epoch 117 train loss 4.120378494262695
val loss 3.7597272396087646
______________
epoch 118 train loss 4.1815032958984375
val loss 3.754011869430542
______________
epoch 119 train loss 4.293508052825928
val loss 3.7484607696533203
______________
epoch 120 train loss 4.0450873374938965
val loss 3.742929697036743
______________
epoch 121 train loss 4.227777004241943
val loss 3.7375850677490234
______________
epoch 122 train loss 4.266199111938477
val loss 3.732427120

epoch 215 train loss 3.658149003982544
val loss 3.454775094985962
______________
epoch 216 train loss 3.580660581588745
val loss 3.452946424484253
______________
epoch 217 train loss 3.718935489654541
val loss 3.451059103012085
______________
epoch 218 train loss 3.6516294479370117
val loss 3.449281930923462
______________
epoch 219 train loss 3.6823983192443848
val loss 3.4474222660064697
______________
epoch 220 train loss 3.617921829223633
val loss 3.4455738067626953
______________
epoch 221 train loss 3.688654661178589
val loss 3.4436748027801514
______________
epoch 222 train loss 3.6806130409240723
val loss 3.441771984100342
______________
epoch 223 train loss 3.7160050868988037
val loss 3.4399399757385254
______________
epoch 224 train loss 3.6901636123657227
val loss 3.4381303787231445
______________
epoch 225 train loss 3.72917103767395
val loss 3.4363410472869873
______________
epoch 226 train loss 3.5759496688842773
val loss 3.4345202445983887
______________
epoch 227 train 

val loss 3.2970492839813232
______________
epoch 320 train loss 3.449652671813965
val loss 3.2961435317993164
______________
epoch 321 train loss 3.439864158630371
val loss 3.295297622680664
______________
epoch 322 train loss 3.362490653991699
val loss 3.294382333755493
______________
epoch 323 train loss 3.387547492980957
val loss 3.2936017513275146
______________
epoch 324 train loss 3.3449220657348633
val loss 3.2927448749542236
______________
epoch 325 train loss 3.3294546604156494
val loss 3.291825532913208
______________
epoch 326 train loss 3.3647406101226807
val loss 3.2908735275268555
______________
epoch 327 train loss 3.3525094985961914
val loss 3.289863109588623
______________
epoch 328 train loss 3.309746265411377
val loss 3.288764715194702
______________
epoch 329 train loss 3.402501344680786
val loss 3.287614583969116
______________
epoch 330 train loss 3.3374152183532715
val loss 3.2864725589752197
______________
epoch 331 train loss 3.407655715942383
val loss 3.285360

val loss 3.2134628295898438
______________
epoch 421 train loss 3.244910717010498
val loss 3.212829351425171
______________
epoch 422 train loss 3.1761891841888428
val loss 3.212137460708618
______________
epoch 423 train loss 3.1550159454345703
val loss 3.211280345916748
______________
epoch 424 train loss 3.1989967823028564
val loss 3.210329294204712
______________
epoch 425 train loss 3.2992453575134277
val loss 3.209467649459839
______________
epoch 426 train loss 3.1627585887908936
val loss 3.2086548805236816
______________
epoch 427 train loss 3.2535293102264404
val loss 3.2078168392181396
______________
epoch 428 train loss 3.2041923999786377
val loss 3.2070600986480713
______________
epoch 429 train loss 3.164231777191162
val loss 3.2062439918518066
______________
epoch 430 train loss 3.1740477085113525
val loss 3.2053277492523193
______________
epoch 431 train loss 3.151127815246582
val loss 3.2044055461883545
______________
epoch 432 train loss 3.2234392166137695
val loss 3.2

val loss 3.1613035202026367
______________
epoch 526 train loss 3.1454951763153076
val loss 3.160902738571167
______________
epoch 527 train loss 3.1057627201080322
val loss 3.1603498458862305
______________
epoch 528 train loss 3.1376500129699707
val loss 3.1599783897399902
______________
epoch 529 train loss 3.0357484817504883
val loss 3.1595938205718994
______________
epoch 530 train loss 3.125753164291382
val loss 3.1591968536376953
______________
epoch 531 train loss 3.1871085166931152
val loss 3.1588706970214844
______________
epoch 532 train loss 3.1742355823516846
val loss 3.1585476398468018
______________
epoch 533 train loss 3.1305651664733887
val loss 3.158234119415283
______________
epoch 534 train loss 3.1239283084869385
val loss 3.1580350399017334
______________
epoch 535 train loss 3.1286399364471436
val loss 3.1577892303466797
______________
epoch 536 train loss 3.1549975872039795
val loss 3.15753436088562
______________
epoch 537 train loss 3.0589611530303955
val loss 

OutcomeSimulator(
  (input_dropout): Dropout(p=0.5, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=72, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=500, bias=True)
    (3): ReLU()
  )
  (batchnorm): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.9, inplace=False)
  (sigmoid): Sigmoid()
  (softmax): LogSoftmax(dim=1)
  (disease_layer): Linear(in_features=500, out_features=3, bias=True)
  (nodal_disease_layer): Linear(in_features=500, out_features=3, bias=True)
  (dlt_layers): ModuleList(
    (0): Linear(in_features=500, out_features=1, bias=True)
    (1): Linear(in_features=500, out_features=1, bias=True)
    (2): Linear(in_features=500, out_features=1, bias=True)
    (3): Linear(in_features=500, out_features=1, bias=True)
    (4): Linear(in_features=500, out_features=1, bias=True)
    (5): Linear(in_features=500, out_features=1, bias=True)
    (6): Linear(in_feat

In [357]:
t3_args = {'hidden_layers': [500, 500], 'dropout': 0.9, 'input_dropout': 0.5}
model3,_,_ = train_state(model_args=t3_args,state=3,use_attention=False,use_smote=False)
model3

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


epoch 0 train loss 2.2618024349212646
val loss 2.143848419189453
______________
epoch 1 train loss 2.2406535148620605
val loss 2.1115522384643555
______________
epoch 2 train loss 2.159592628479004
val loss 2.080714702606201
______________
epoch 3 train loss 2.0866901874542236
val loss 2.0511999130249023
______________
epoch 4 train loss 2.091585636138916
val loss 2.023052453994751
______________
epoch 5 train loss 2.0663974285125732
val loss 1.9962196350097656
______________
epoch 6 train loss 1.9959940910339355
val loss 1.9704458713531494
______________
epoch 7 train loss 1.9626026153564453
val loss 1.9457974433898926
______________
epoch 8 train loss 1.9203476905822754
val loss 1.9220383167266846
______________
epoch 9 train loss 1.8915472030639648
val loss 1.8990752696990967
______________
epoch 10 train loss 1.8611359596252441
val loss 1.8769221305847168
______________
epoch 11 train loss 1.862541675567627
val loss 1.8556615114212036
______________
epoch 12 train loss 1.8137745857

val loss 1.3732812404632568
______________
epoch 104 train loss 1.3399834632873535
val loss 1.3712007999420166
______________
epoch 105 train loss 1.3930026292800903
val loss 1.369257926940918
______________
epoch 106 train loss 1.3601747751235962
val loss 1.367357850074768
______________
epoch 107 train loss 1.3442729711532593
val loss 1.3654862642288208
______________
epoch 108 train loss 1.3501203060150146
val loss 1.3636412620544434
______________
epoch 109 train loss 1.377656102180481
val loss 1.361830711364746
______________
epoch 110 train loss 1.371109962463379
val loss 1.3601478338241577
______________
epoch 111 train loss 1.352440595626831
val loss 1.3584325313568115
______________
epoch 112 train loss 1.311982274055481
val loss 1.3567085266113281
______________
epoch 113 train loss 1.3158514499664307
val loss 1.3548954725265503
______________
epoch 114 train loss 1.3635830879211426
val loss 1.3531363010406494
______________
epoch 115 train loss 1.3940324783325195
val loss 1.

val loss 1.2647268772125244
______________
epoch 213 train loss 1.2452644109725952
val loss 1.2643617391586304
______________
epoch 214 train loss 1.242777943611145
val loss 1.2640572786331177
______________
epoch 215 train loss 1.233198881149292
val loss 1.2637630701065063
______________
epoch 216 train loss 1.2229982614517212
val loss 1.2634495496749878
______________
epoch 217 train loss 1.2289142608642578
val loss 1.263178825378418
______________
epoch 218 train loss 1.2482945919036865
val loss 1.2629578113555908
______________
epoch 219 train loss 1.237855076789856
val loss 1.262730360031128
______________
epoch 220 train loss 1.241115927696228
val loss 1.2625415325164795
______________
epoch 221 train loss 1.2059739828109741
val loss 1.2623170614242554
______________
epoch 222 train loss 1.2172269821166992
val loss 1.2620867490768433
______________
epoch 223 train loss 1.2195500135421753
val loss 1.2618577480316162
______________
epoch 224 train loss 1.2369847297668457
val loss 1

val loss 1.2408045530319214
______________
epoch 321 train loss 1.183243751525879
val loss 1.2407293319702148
______________
epoch 322 train loss 1.164512276649475
val loss 1.2406764030456543
______________
epoch 323 train loss 1.198918342590332
val loss 1.2406634092330933
______________
epoch 324 train loss 1.1977900266647339
val loss 1.2406835556030273
______________
epoch 325 train loss 1.137998342514038
val loss 1.2407160997390747
______________
epoch 326 train loss 1.1934255361557007
val loss 1.2407705783843994
______________
epoch 327 train loss 1.186288595199585
val loss 1.2408310174942017
______________
epoch 328 train loss 1.161870002746582
val loss 1.2408814430236816
______________
epoch 329 train loss 1.201947569847107
val loss 1.2408894300460815
______________
epoch 330 train loss 1.135907769203186
val loss 1.2408902645111084
______________
epoch 331 train loss 1.1495435237884521
val loss 1.2408567667007446
______________
epoch 332 train loss 1.1719534397125244
val loss 1.2

EndpointSimulator(
  (input_dropout): Dropout(p=0.5, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=70, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=500, bias=True)
    (3): ReLU()
  )
  (batchnorm): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.9, inplace=False)
  (sigmoid): Sigmoid()
  (softmax): LogSoftmax(dim=1)
  (outcome_layer): Linear(in_features=500, out_features=3, bias=True)
)

In [156]:
def gridsearch_attention_transition_models(state=1,attentions=[True,False]):
    model_arglist = [
        {
            'hidden_layers': [100],
            'attention_heads': [1],
        },
        {
            'hidden_layers': [500],
            'attention_heads': [1],
        },
        {
            'hidden_layers': [1000],
            'attention_heads': [10],
        },
        {
            'hidden_layers': [100],
            'attention_heads': [5],
        },
        {
            'hidden_layers': [500],
            'attention_heads': [5],
        },
        {
            'hidden_layers': [1000],
            'attention_heads': [5],
        },
        {
            'hidden_layers': [100,100],
            'attention_heads': [10,10],
        },
        {
            'hidden_layers': [500,500],
            'attention_heads': [10,10],
        },
        {
            'hidden_layers': [100,100],
            'attention_heads': [5,5],
        },
        {
            'hidden_layers': [500,500],
            'attention_heads': [5,5],
        },
    ]
    best_loss = 100000000000
    best_metrics = {}
    best_args = {}
    best_model = None
    k = 0
    for margs in model_arglist:
        args = {k:v for k,v in margs.items()}
        for use_attention in attentions:
            embed_sizes = [200,400] if use_attention else [0]
            for embed_size in embed_sizes:
                args['embed_size'] = embed_size
                for dropout in [.9,]:
                    args['dropout'] = dropout
                    for input_dropout in [.5]:
                        args['input_dropout'] = input_dropout
                    
                        model,m_loss,m_metrics = train_state(model_args=args,state=state,use_attention=use_attention,verbose=False)
                        print('done',k,m_loss)
                        k+=1
                        if m_loss < best_loss:
                            best_loss = m_loss
                            best_metrics  = m_metrics
                            best_model = model
                            best_args = args
                            print('_++++++++++New Best++++____')
                            print(best_loss)
                            print(best_metrics)
                            print(best_args)
                            print(use_attention)
                            print('___________')
                            print('++++++++')
                            print()
    print('_________')
    print('+++++++++++')
    print('best stuff',best_loss)
    print(best_metrics)
    print(best_args)
    return best_model
# model1 = gridsearch_attention_transition_models(1)
# model1

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 2.764965534210205 {'pd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6839781628293378, 'roc_macro': 0.6330226813097114}, 'nd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6238666993383974, 'roc_macro': 0.6753406617537051}, 'mod': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6238666993383974, 'roc_macro': 0.6753406617537051}, 'dlts': {'accuracy': [0.9794520547945206, 0.9246575342465754, 0.9178082191780822, 1.0, 0.9657534246575342, 1.0, 0.958904109589041, 1.0], 'accuracy_mean': 0.9683219178082192, 'auc': [0.9510489510489512, 0.9313131313131313, 0.9527363184079602, -1, 0.9092198581560285, -1, 0.9333333333333333, -1], 'auc_mean': 0.20970644903242558}}
done 0 2.764965534210205
_++++++++++New Best++++____
2.764965534210205
{'pd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6839781628293378, 'roc_macro': 0.6330226813097114}, 'nd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6238666993383974, 'roc_macro': 0.6753406617537051}, 'mod': {'accuracy': 0.333333333333

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 4.025295257568359 {'pd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6755281272252552, 'roc_macro': 0.6340827355168788}, 'nd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.5679000245037981, 'roc_macro': 0.559701470027557}, 'mod': {'accuracy': 0.3333333333333333, 'roc_micro': 0.5679000245037981, 'roc_macro': 0.559701470027557}, 'dlts': {'accuracy': [0.9794520547945206, 0.9246575342465754, 0.9178082191780822, 1.0, 0.9657534246575342, 1.0, 0.958904109589041, 1.0], 'accuracy_mean': 0.9683219178082192, 'auc': [0.766899766899767, 0.8424242424242425, 0.7170398009950248, -1, 0.75177304964539, -1, 0.8880952380952382, -1], 'auc_mean': 0.12077901225745781}}
done 1 4.025295257568359


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


...

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 2.7161152362823486 {'pd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6835509138381202, 'roc_macro': 0.6361547246381728}, 'nd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6163195295270767, 'roc_macro': 0.6741712393886307}, 'mod': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6163195295270767, 'roc_macro': 0.6741712393886307}, 'dlts': {'accuracy': [0.9794520547945206, 0.9246575342465754, 0.9178082191780822, 1.0, 0.9657534246575342, 1.0, 0.958904109589041, 1.0], 'accuracy_mean': 0.9683219178082192, 'auc': [0.9137529137529138, 0.9683501683501683, 0.9359452736318408, -1, 0.9304964539007092, -1, 0.9011904761904762, -1], 'auc_mean': 0.20621691072826354}}
done 28 2.7161152362823486


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 2.3858258724212646 {'pd': {'accuracy': 0.4969654528478058, 'roc_micro': 0.6851174934725849, 'roc_macro': 0.6237181607808212}, 'nd': {'accuracy': 0.5198805594845198, 'roc_micro': 0.7011026709139917, 'roc_macro': 0.6828778105952019}, 'mod': {'accuracy': 0.5198805594845198, 'roc_micro': 0.7011026709139917, 'roc_macro': 0.6828778105952019}, 'dlts': {'accuracy': [0.9794520547945206, 0.9246575342465754, 0.9178082191780822, 1.0, 0.9657534246575342, 1.0, 0.958904109589041, 1.0], 'accuracy_mean': 0.9683219178082192, 'auc': [0.8951048951048951, 0.9555555555555556, 0.9253731343283582, -1, 0.9432624113475176, -1, 0.8952380952380953, -1], 'auc_mean': 0.20181676144680272}}
done 29 2.3858258724212646
_________
+++++++++++
best stuff 2.251129150390625
{'pd': {'accuracy': 0.375, 'roc_micro': 0.6783764538333729, 'roc_macro': 0.6248571736860679}, 'nd': {'accuracy': 0.4801980198019802, 'roc_micro': 0.6757167360940945, 'roc_macro': 0.6780191426930556}, 'mod': {'accuracy': 0.4801980198019802,

OutcomeAttentionSimulator(
  (input_dropout): Dropout(p=0.5, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=100, out_features=1000, bias=True)
  )
  (batchnorm): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.9, inplace=False)
  (sigmoid): Sigmoid()
  (softmax): LogSoftmax(dim=1)
  (resize_layer): Linear(in_features=63, out_features=100, bias=True)
  (attentions): ModuleList(
    (0): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
    )
  )
  (norms): ModuleList(
    (0): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
  )
  (final_layer): Linear(in_features=1000, out_features=3, bias=True)
  (activation): ReLU()
  (disease_layer): Linear(in_features=1000, out_features=3, bias=True)
  (nodal_disease_layer): Linear(in_features=1000, out_features=3, bias=True)
  (dlt_layers): ModuleList(
    (0): Linear(in_features=1000, out_features=1

In [157]:
# model2 = gridsearch_atten/.tion_transition_models(2)

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains clas

best loss 3.808809757232666 {'pd': {'accuracy': 0.5, 'roc_micro': 0.9072182055232902, 'roc_macro': -1}, 'nd': {'accuracy': 0.350210970464135, 'roc_micro': 0.7220952335121871, 'roc_macro': 0.47419713120932633}, 'mod': {'accuracy': 0.350210970464135, 'roc_micro': 0.7220952335121871, 'roc_macro': 0.47419713120932633}, 'dlts': {'accuracy': [0.9794520547945206, 0.9178082191780822, 0.952054794520548, 1.0, 0.9657534246575342, 1.0, 0.9657534246575342, 1.0], 'accuracy_mean': 0.9726027397260274, 'auc': [0.2564102564102564, 0.5926616915422885, 0.60431654676259, -1, 0.5035460992907801, -1, 0.5943262411347519, -1], 'auc_mean': -0.05609239560741665}}
done 0 3.808809757232666
_++++++++++New Best++++____
3.808809757232666
{'pd': {'accuracy': 0.5, 'roc_micro': 0.9072182055232902, 'roc_macro': -1}, 'nd': {'accuracy': 0.350210970464135, 'roc_micro': 0.7220952335121871, 'roc_macro': 0.47419713120932633}, 'mod': {'accuracy': 0.350210970464135, 'roc_micro': 0.7220952335121871, 'roc_macro': 0.474197131209326

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


...

best loss 3.8991506099700928 {'pd': {'accuracy': 0.5, 'roc_micro': 0.9260874718501837, 'roc_macro': -1}, 'nd': {'accuracy': 0.32383966244725737, 'roc_micro': 0.7193041553379371, 'roc_macro': 0.484346802563266}, 'mod': {'accuracy': 0.32383966244725737, 'roc_micro': 0.7193041553379371, 'roc_macro': 0.484346802563266}, 'dlts': {'accuracy': [0.9794520547945206, 0.9178082191780822, 0.952054794520548, 1.0, 0.9657534246575342, 1.0, 0.9657534246575342, 1.0], 'accuracy_mean': 0.9726027397260274, 'auc': [0.48717948717948717, 0.5690298507462687, 0.4696813977389517, -1, 0.5397163120567376, -1, 0.3588652482269504, -1], 'auc_mean': -0.07194096300645057}}
done 28 3.8991506099700928


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains clas

best loss 3.4415688514709473 {'pd': {'accuracy': 0.4959349593495935, 'roc_micro': 0.9374659239066019, 'roc_macro': -1}, 'nd': {'accuracy': 0.34236550632911394, 'roc_micro': 0.7523639950915522, 'roc_macro': 0.6504770843566575}, 'mod': {'accuracy': 0.34236550632911394, 'roc_micro': 0.7523639950915522, 'roc_macro': 0.6504770843566575}, 'dlts': {'accuracy': [0.9794520547945206, 0.9178082191780822, 0.952054794520548, 1.0, 0.9657534246575342, 1.0, 0.9657534246575342, 1.0], 'accuracy_mean': 0.9726027397260274, 'auc': [0.3776223776223776, 0.595771144278607, 0.6402877697841727, -1, 0.6624113475177306, -1, 0.3801418439716312, -1], 'auc_mean': -0.04297068960318512}}
done 29 3.4415688514709473
_++++++++++New Best++++____
3.4415688514709473
{'pd': {'accuracy': 0.4959349593495935, 'roc_micro': 0.9374659239066019, 'roc_macro': -1}, 'nd': {'accuracy': 0.34236550632911394, 'roc_micro': 0.7523639950915522, 'roc_macro': 0.6504770843566575}, 'mod': {'accuracy': 0.34236550632911394, 'roc_micro': 0.75236399

In [158]:
# model3 = gridsearch_attention_transition_models(3)

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 1.9533352851867676 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.22294936, 'auc': 0.61875}, 'FT': {'accuracy': -1, 'mse': 0.22988118, 'auc': 0.6208881578947368}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.23439899, 'auc': 0.7096153846153845}}
done 0 1.9533352851867676
_++++++++++New Best++++____
1.9533352851867676
{'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.22294936, 'auc': 0.61875}, 'FT': {'accuracy': -1, 'mse': 0.22988118, 'auc': 0.6208881578947368}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.23439899, 'auc': 0.7096153846153845}}
{'hidden_layers': [100], 'attention_heads': [1], 'embed_size': 200, 'dropout': 0.9, 'input_dropout': 0.5}
True
___________
++++++++



/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 1.9169039726257324 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.23428532, 'auc': 0.5783653846153846}, 'FT': {'accuracy': -1, 'mse': 0.21831562, 'auc': 0.6165021929824561}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.21789795, 'auc': 0.7125}}
done 1 1.9169039726257324
_++++++++++New Best++++____
1.9169039726257324
{'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.23428532, 'auc': 0.5783653846153846}, 'FT': {'accuracy': -1, 'mse': 0.21831562, 'auc': 0.6165021929824561}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.21789795, 'auc': 0.7125}}
{'hidden_layers': [100], 'attention_heads': [1], 'embed_size': 400, 'dropout': 0.9, 'input_dropout': 0.5}
True
___________
++++++++



/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


...

here
best loss 1.513304352760315 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.15192407, 'auc': 0.6110576923076922}, 'FT': {'accuracy': -1, 'mse': 0.19458365, 'auc': 0.5923793859649122}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.14482027, 'auc': 0.7080128205128204}}
done 27 1.513304352760315


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 1.455763339996338 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.12546952, 'auc': 0.6596153846153847}, 'FT': {'accuracy': -1, 'mse': 0.20594049, 'auc': 0.6058114035087719}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.14230496, 'auc': 0.7096153846153845}}
done 28 1.455763339996338
_++++++++++New Best++++____
1.455763339996338
{'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.12546952, 'auc': 0.6596153846153847}, 'FT': {'accuracy': -1, 'mse': 0.20594049, 'auc': 0.6058114035087719}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.14230496, 'auc': 0.7096153846153845}}
{'hidden_layers': [500, 500], 'attention_heads': [5, 5], 'embed_size': 400, 'dropout': 0.9, 'input_dropout': 0.5}
True
___________
++++++++



/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 1.4676107168197632 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.12858523, 'auc': 0.6384615384615384}, 'FT': {'accuracy': -1, 'mse': 0.19938947, 'auc': 0.6077302631578947}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.13879225, 'auc': 0.7153846153846154}}
done 29 1.4676107168197632
_________
+++++++++++
best stuff 1.455763339996338
{'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.12546952, 'auc': 0.6596153846153847}, 'FT': {'accuracy': -1, 'mse': 0.20594049, 'auc': 0.6058114035087719}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.14230496, 'auc': 0.7096153846153845}}
{'hidden_layers': [500, 500], 'attention_heads': [5, 5], 'embed_size': 0, 'dropout': 0.9, 'input_dropout': 0.5}


In [ ]:
def load_trained_models():
    files = Const.tuned_transition_models
    decision_file = Const.tuned_decision_model
    [model1,model2,model3] = [torch.load(file) for file in files]
    decision_model = torch.load(decision_file)
    return decision_model, model1,model2,model3
_, model1, model2, model3 =load_trained_models()

In [352]:

def shuffle_col(v,col=None):
    if col is None:
        col = np.random.choice([i for i in range(v.shape[1])])
    idx = torch.randperm(v.shape[0])
    vv = torch.clone(v)
    vv[:,col] = vv[idx,col]
    return vv
    
def train_decision_model(
    tmodel1,
    tmodel2,
    tmodel3,
    use_default_split=True,
    use_bagging_split=False,
    lr=.0001,
    epochs=10000,
    patience=100,
    weights=[3,1,1], #realtive weight of survival, feeding tube, and aspiration
    imitation_weight=1,
    shufflecol_chance = 0.1,
    reward_weight=10,
    split=.7,
    resample_training=False,
    save_path='../data/models/',
    file_suffix='',
    use_attention=False,
    verbose=True,
    threshold_decisions=True,
    use_smote=True,
    **model_kwargs,
):
    
    tmodel1.eval()
    tmodel2.eval()
    tmodel3.eval()
    
    train_ids, test_ids = get_tt_split(use_default_split=use_default_split,use_bagging_split=use_bagging_split,resample_training=resample_training)

    if use_smote:
        dataset = DTDataset(use_smote=True,smote_ids = train_ids)
        train_ids = [i for i in dataset.processed_df.index.values if i not in test_ids]
    else:
        dataset = DTDataset()

    data = dataset.processed_df.copy()
    
    
    def get_dlt(state):
        if state == 2:
            return data[Const.dlt2].copy()
        d = data[Const.dlt1].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_pd(state):
        if state == 2:
            return data[Const.primary_disease_states2].copy()
        d = data[Const.primary_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_nd(state):
        if state == 2:
            return data[Const.nodal_disease_states2].copy()
        d = data[Const.nodal_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_cc(state):
        res = data[Const.ccs].copy()
        if state == 1:
            res.values[:,:] = np.zeros(res.values.shape)
        return res
    
    def get_mod(state):
        res = data[Const.modifications].copy()
        return res
        
    outcomedf = data[Const.outcomes]
    baseline = dataset.get_state('baseline')
    
    def formatdf(d,dids=train_ids):
        d = df_to_torch(d.loc[dids])
        return d
    
    def makegrad(v):
        if not v.requires_grad:
            v.requires_grad=True
        return v
    
    if use_attention:
        model = DecisionAttentionModel(baseline.shape[1],**model_kwargs)
    else:
        model = DecisionModel(baseline.shape[1],**model_kwargs)

    hashcode = str(hash(','.join([str(i) for i in train_ids])))
    
    save_file = save_path + 'model_' + model.identifier +'_hash' + hashcode + file_suffix + '.tar'
    model.fit_normalizer(df_to_torch(baseline.loc[train_ids]))
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)

    def outcome_loss(ypred):
        #convert survival to death
        loss = torch.mul(torch.mean(-1*(ypred[:,0] - 1)),weights[0])
        for i,weight in enumerate(weights[1:]):
            newloss = torch.mean(ypred[:,i])*weight
            loss = torch.add(loss,torch.mul(newloss,weight))
        return loss
    
    mse = torch.nn.MSELoss()
    nllloss = torch.nn.NLLLoss()
    bce = torch.nn.BCELoss()
    
    def compare_decisions(d1,d2,d3,ids):
#         ypred = np.concatenate([dd.cpu().detach().numpy().reshape(-1,1) for dd in [d1,d2,d3]],axis=1)
        ytrue = df_to_torch(outcomedf.loc[ids])
        dloss = bce(d1.view(-1),ytrue[:,0])
        dloss += bce(d2.view(-1),ytrue[:,1])
        dloss += bce(d3,view(-1),ytrue[:,2])
        return dloss
        
    def remove_decisions(df):
        cols = [c for c in df.columns if c not in Const.decisions ]
        ddf = df[cols]
        return ddf
    
    makeinput = lambda step,dids: df_to_torch(remove_decisions(dataset.get_input_state(step=step,ids=dids)))
    threshold = lambda x: torch.gt(x,.5).type(torch.FloatTensor)

    
    #save the inputs from the whole dataset for future reference
    full_data = []
    for mstep in [0,1,2]:
        full_data_step = [baseline, get_dlt(min(mstep,1)),
                     get_dlt(mstep),get_pd(mstep),get_nd(mstep),get_cc(mstep),get_mod(mstep)]
        full_data_step = torch.cat([formatdf(fd,train_ids+test_ids) for fd in full_data_step],axis=1)
        full_data.append(full_data_step)
    full_data = torch.stack(full_data)
    model.save_memory(full_data)

    def step(train=True):
        if train:
            model.train(True)
            optimizer.zero_grad()
            ids = train_ids
        else:
            ids = test_ids
            model.eval()
            
            
        ytrain = df_to_torch(outcomedf.loc[ids])
        #imitation losses and decision 1
        xxtrained = [baseline, get_dlt(0),get_dlt(0),get_pd(0),get_nd(0),get_cc(0),get_mod(0)]
        xxtrain = [formatdf(xx,ids) for xx in xxtrained]
        o1 = model(torch.cat(xxtrain,axis=1),position=0)
            
        decision1_imitation = o1[:,3]
        decision1 = o1[:,0]
        if threshold_decisions:
            decision1 = threshold(decision1)
#         imitation_loss1 = bce(threshold(decision1_imitation),ytrain[:,0])
        imitation_loss1 = bce(decision1_imitation,ytrain[:,0])
        
        x1_imitation = [baseline, get_dlt(1),get_dlt(0),get_pd(1),get_nd(1),get_cc(1),get_mod(1)]
        x1_imitation = [formatdf(xx1,ids) for xx1 in x1_imitation]
        decision2_imitation = model(torch.cat(x1_imitation,axis=1),position=1)[:,4]
        
#         imitation_loss2 =  bce(threshold(decision2_imitation),ytrain[:,1])
        imitation_loss2 =  bce(decision2_imitation,ytrain[:,1])
        
        x2_imitation = [baseline, get_dlt(1),get_dlt(2),get_pd(2),get_nd(2),get_cc(2),get_mod(2)]
#         if model.keys is None:
#             keyvals = torch.cat([formatdf(xxx,train_ids+test_ids) for xxx in x2_imitation],axis=1)
#             model.set_keys(keyvals)
            
        x2_imitation = [formatdf(xx2,ids) for xx2 in x2_imitation]
        
        decision3_imitation = model(torch.cat(x2_imitation,axis=1),position=2)[:,5]
        
#         imitation_loss3 = bce(threshold(decision3_imitation),ytrain[:,2])
        imitation_loss3 = bce(decision3_imitation,ytrain[:,2])
        
        #reward decisions
        xx1 = makeinput(1,ids)
        xx2 = makeinput(2,ids)
        xx3 = makeinput(3,ids)

        baseline_train_base = formatdf(baseline,ids)
            
        baseline_train = torch.clone(baseline_train_base)
        if train and shufflecol_chance > 0.0001:
            for col in range(baseline_train_base.shape[1]): 
                if np.random.random() < shufflecol_chance:
                    baseline_train = shuffle_col(baseline_train,col)
                    
        
        xi1 = torch.cat([xx1,decision1.view(-1,1)],axis=1)
        [ypd1, ynd1, ymod, ydlt1] = tmodel1(xi1)
        #this outputs log likelihoods (except for dlts) -> convert to probability
        ypd1 = torch.exp(ypd1)
        ynd1 = torch.exp(ynd1)
        ymod = torch.exp(ymod)
        x1 = [baseline_train,ydlt1,formatdf(get_dlt(0),ids),ypd1,ynd1,formatdf(get_cc(1),ids),ymod]
        
        decision2 = model(torch.cat(x1,axis=1),position=1)[:,1] 
        if threshold_decisions:
            decision2 = threshold(decision2)
            
        xi2 = torch.cat([xx2,decision1.view(-1,1),decision2.view(-1,1)],axis=1)
        [ypd2,ynd2,ycc,ydlt2] = tmodel2(xi2)
        ypd2 = torch.exp(ypd2)
        ynd2 = torch.exp(ynd2)
        ycc = torch.exp(ycc)
        x2 = [baseline_train,ydlt1,ydlt2,ypd2,ynd2,ycc,ymod]
            
        decision3 = model(torch.cat(x2,axis=1),position=2)[:,2]
        if threshold_decisions:
            decision3 = threshold(decision3)
            
        
        xi3 = torch.cat([xx3,decision1.view(-1,1),decision2.view(-1,1),decision3.view(-1,1)],axis=1)
        outcomes = tmodel3(xi3)

        reward_loss = outcome_loss(outcomes)
        loss = torch.add(imitation_loss1,imitation_loss2)
        loss = torch.add(loss,imitation_loss3)
        loss = torch.mul(loss,imitation_weight/3)
        loss = torch.add(loss,torch.mul(reward_loss,reward_weight))
        losses = [imitation_loss1+imitation_loss2+imitation_loss3,reward_loss]
        if train:
            loss.backward()
            optimizer.step()
            return losses
        else:
            scores = []
            for i,decision in enumerate([decision1_imitation,decision2_imitation,decision3_imitation]):
                dec = decision.cpu().detach().numpy()
                dec0 = (dec > .5).astype(int)
                out = ytrain[:,i].cpu().detach().numpy()
                acc = accuracy_score(out,dec > .5)
                auc = roc_auc_score(out,dec)
                scores.append({'decision': i,'accuracy': acc,'auc': auc})
            return losses, scores
        
    best_val_loss = torch.tensor(1000000000.0)
    steps_since_improvement = 0
    best_val_score = {}
    for epoch in range(epochs):
        losses = step(True)
        val_losses,val_metrics = step(False)
        vl = val_losses[0] + val_losses[1]
        if verbose:
            print('______epoch',str(epoch),'_____')
            print('train imitation',losses[0].item(),'reward',losses[1].item())
            print('val imitation',val_losses[0].item(),'reward',val_losses[1].item())
            print('val loss',vl.item(),best_val_loss.item())
            print(val_metrics)
        if vl < best_val_loss:
            best_val_loss = vl
            best_val_score = val_metrics
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
        if steps_since_improvement > patience:
            break
    print('++++++++++Final+++++++++++')
    print('best',best_val_loss)
    print(best_val_score)
    model.load_state_dict(torch.load(save_file))
    model.eval()
    return model, best_val_score, best_val_loss

from Models import *
args = {
    'hidden_layers': [600,300], 
    'attention_heads': [3,1], 
    'embed_size': 210, 
    'dropout': 0.9, 
    'input_dropout': 0.5, 
    'shufflecol_chance': 0.5,
}
decision_model, _, _ = train_decision_model(model1,model2,model3,lr=.0001,use_attention=True,**args)

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
______epoch 0 _____
train imitation 2.206021547317505 reward 2.491159677505493
val imitation 2.1344025135040283 reward 1.8243277072906494
val loss 3.9587302207946777 1000000000.0
[{'decision': 0, 'accuracy': 0.3356164383561644, 'auc': 0.6951923076923078}, {'decision': 1, 'accuracy': 0.363013698630137, 'auc': 0.48300438596491224}, {'decision': 2, 'accuracy': 0.5136986301369864, 'auc': 0.6221153846153846}]
______epoch 1 _____
train imitation 2.191789150238037 reward 2.4916346073150635
val imitation 2.09209942817688 reward 1.8282902240753174
val loss 3.9203896522521973 3.9587302207946777
[{'decision': 0, 'accuracy': 0.3698630136986301, 'auc': 0.701923076923077}, {'decision': 1, 'accuracy': 0.4315068493150685, 'auc': 0.5098684210526316}, {'decision': 2, 'accuracy': 0.6232876712328768, 'auc': 0.6842948717948718}]
______epoch 2 _____
train imitation 2.174431562423706 reward 2.4919028282165527
val imitation 2.0585765838623047 reward 1.8293774127960205
val loss 3.887953996658325 3.9203896

______epoch 20 _____
train imitation 2.0111188888549805 reward 2.493630886077881
val imitation 1.930152177810669 reward 1.833322286605835
val loss 3.763474464416504 3.7719199657440186
[{'decision': 0, 'accuracy': 0.7054794520547946, 'auc': 0.6153846153846154}, {'decision': 1, 'accuracy': 0.6438356164383562, 'auc': 0.6554276315789473}, {'decision': 2, 'accuracy': 0.7945205479452054, 'auc': 0.8358974358974358}]
______epoch 21 _____
train imitation 2.013956308364868 reward 2.493551731109619
val imitation 1.9202048778533936 reward 1.8355802297592163
val loss 3.7557849884033203 3.763474464416504
[{'decision': 0, 'accuracy': 0.7191780821917808, 'auc': 0.6134615384615385}, {'decision': 1, 'accuracy': 0.6506849315068494, 'auc': 0.6559758771929824}, {'decision': 2, 'accuracy': 0.7945205479452054, 'auc': 0.8349358974358975}]
______epoch 22 _____
train imitation 2.013810157775879 reward 2.4933156967163086
val imitation 1.9114313125610352 reward 1.836445689201355
val loss 3.7478771209716797 3.7557

______epoch 40 _____
train imitation 1.9267195463180542 reward 2.496753454208374
val imitation 1.759631872177124 reward 1.8455004692077637
val loss 3.6051323413848877 3.612441301345825
[{'decision': 0, 'accuracy': 0.7671232876712328, 'auc': 0.575}, {'decision': 1, 'accuracy': 0.6643835616438356, 'auc': 0.6759868421052633}, {'decision': 2, 'accuracy': 0.7945205479452054, 'auc': 0.823076923076923}]
______epoch 41 _____
train imitation 1.9307422637939453 reward 2.4959752559661865
val imitation 1.7474699020385742 reward 1.846486210823059
val loss 3.5939559936523438 3.6051323413848877
[{'decision': 0, 'accuracy': 0.7602739726027398, 'auc': 0.5750000000000001}, {'decision': 1, 'accuracy': 0.6712328767123288, 'auc': 0.6759868421052632}, {'decision': 2, 'accuracy': 0.7808219178082192, 'auc': 0.8227564102564102}]
______epoch 42 _____
train imitation 1.9262444972991943 reward 2.4975411891937256
val imitation 1.7365131378173828 reward 1.844804286956787
val loss 3.58131742477417 3.5939559936523438

______epoch 60 _____
train imitation 1.853175163269043 reward 2.498384475708008
val imitation 1.583315372467041 reward 1.8504291772842407
val loss 3.433744430541992 3.436608076095581
[{'decision': 0, 'accuracy': 0.7876712328767124, 'auc': 0.5615384615384615}, {'decision': 1, 'accuracy': 0.6917808219178082, 'auc': 0.6880482456140351}, {'decision': 2, 'accuracy': 0.7945205479452054, 'auc': 0.8173076923076923}]
______epoch 61 _____
train imitation 1.8535605669021606 reward 2.4979989528656006
val imitation 1.5808353424072266 reward 1.8510133028030396
val loss 3.4318485260009766 3.433744430541992
[{'decision': 0, 'accuracy': 0.7808219178082192, 'auc': 0.5615384615384615}, {'decision': 1, 'accuracy': 0.6917808219178082, 'auc': 0.6880482456140351}, {'decision': 2, 'accuracy': 0.7945205479452054, 'auc': 0.8176282051282051}]
______epoch 62 _____
train imitation 1.8503973484039307 reward 2.497927188873291
val imitation 1.576523780822754 reward 1.8510133028030396
val loss 3.427536964416504 3.4318

...

______epoch 120 _____
train imitation 1.701364278793335 reward 2.4978904724121094
val imitation 1.4330124855041504 reward 1.8532582521438599
val loss 3.2862706184387207 3.2530345916748047
[{'decision': 0, 'accuracy': 0.7876712328767124, 'auc': 0.5605769230769231}, {'decision': 1, 'accuracy': 0.6986301369863014, 'auc': 0.6992872807017543}, {'decision': 2, 'accuracy': 0.815068493150685, 'auc': 0.8317307692307692}]
______epoch 121 _____
train imitation 1.680197834968567 reward 2.4978318214416504
val imitation 1.4306070804595947 reward 1.853963851928711
val loss 3.2845709323883057 3.2530345916748047
[{'decision': 0, 'accuracy': 0.7876712328767124, 'auc': 0.5600961538461539}, {'decision': 1, 'accuracy': 0.6986301369863014, 'auc': 0.6984649122807017}, {'decision': 2, 'accuracy': 0.815068493150685, 'auc': 0.832051282051282}]
______epoch 122 _____
train imitation 1.694934368133545 reward 2.4960415363311768
val imitation 1.4234720468521118 reward 1.8540376424789429
val loss 3.2775096893310547 3

______epoch 140 _____
train imitation 1.6135034561157227 reward 2.4983630180358887
val imitation 1.4536811113357544 reward 1.8471031188964844
val loss 3.300784111022949 3.2530345916748047
[{'decision': 0, 'accuracy': 0.7397260273972602, 'auc': 0.5524038461538462}, {'decision': 1, 'accuracy': 0.6986301369863014, 'auc': 0.7012061403508772}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.8294871794871794}]
______epoch 141 _____
train imitation 1.6265554428100586 reward 2.4953114986419678
val imitation 1.449310541152954 reward 1.846771478652954
val loss 3.296082019805908 3.2530345916748047
[{'decision': 0, 'accuracy': 0.7534246575342466, 'auc': 0.5514423076923077}, {'decision': 1, 'accuracy': 0.684931506849315, 'auc': 0.7012061403508771}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.828525641025641}]
______epoch 142 _____
train imitation 1.661378264427185 reward 2.4976086616516113
val imitation 1.4348303079605103 reward 1.8482502698898315
val loss 3.283080577850342 3.25

______epoch 160 _____
train imitation 1.6117401123046875 reward 2.4976818561553955
val imitation 1.4917644262313843 reward 1.846806526184082
val loss 3.338571071624756 3.2530345916748047
[{'decision': 0, 'accuracy': 0.6917808219178082, 'auc': 0.5408653846153846}, {'decision': 1, 'accuracy': 0.6643835616438356, 'auc': 0.6998355263157895}, {'decision': 2, 'accuracy': 0.815068493150685, 'auc': 0.8262820512820512}]
______epoch 161 _____
train imitation 1.5723929405212402 reward 2.4961791038513184
val imitation 1.484464406967163 reward 1.8462791442871094
val loss 3.3307435512542725 3.2530345916748047
[{'decision': 0, 'accuracy': 0.7054794520547946, 'auc': 0.5384615384615385}, {'decision': 1, 'accuracy': 0.6643835616438356, 'auc': 0.7006578947368421}, {'decision': 2, 'accuracy': 0.815068493150685, 'auc': 0.8262820512820512}]
______epoch 162 _____
train imitation 1.5858948230743408 reward 2.496967315673828
val imitation 1.4792784452438354 reward 1.8465592861175537
val loss 3.3258376121520996 

______epoch 180 _____
train imitation 1.5430561304092407 reward 2.496851682662964
val imitation 1.5471850633621216 reward 1.8474061489105225
val loss 3.3945913314819336 3.2530345916748047
[{'decision': 0, 'accuracy': 0.684931506849315, 'auc': 0.5346153846153846}, {'decision': 1, 'accuracy': 0.6506849315068494, 'auc': 0.6959978070175439}, {'decision': 2, 'accuracy': 0.8013698630136986, 'auc': 0.8237179487179488}]
______epoch 181 _____
train imitation 1.5376200675964355 reward 2.4966955184936523
val imitation 1.5559942722320557 reward 1.8463497161865234
val loss 3.402343988418579 3.2530345916748047
[{'decision': 0, 'accuracy': 0.684931506849315, 'auc': 0.5331730769230769}, {'decision': 1, 'accuracy': 0.6438356164383562, 'auc': 0.694078947368421}, {'decision': 2, 'accuracy': 0.8013698630136986, 'auc': 0.8233974358974359}]
______epoch 182 _____
train imitation 1.5171308517456055 reward 2.4969708919525146
val imitation 1.559504747390747 reward 1.8463497161865234
val loss 3.4058544635772705 

______epoch 200 _____
train imitation 1.5286964178085327 reward 2.495995283126831
val imitation 1.691309928894043 reward 1.8476234674453735
val loss 3.538933277130127 3.2530345916748047
[{'decision': 0, 'accuracy': 0.6575342465753424, 'auc': 0.5399038461538461}, {'decision': 1, 'accuracy': 0.6232876712328768, 'auc': 0.6842105263157895}, {'decision': 2, 'accuracy': 0.8013698630136986, 'auc': 0.8125}]
______epoch 201 _____
train imitation 1.493225336074829 reward 2.4958553314208984
val imitation 1.7113475799560547 reward 1.847294807434082
val loss 3.5586423873901367 3.2530345916748047
[{'decision': 0, 'accuracy': 0.6506849315068494, 'auc': 0.5403846153846155}, {'decision': 1, 'accuracy': 0.6301369863013698, 'auc': 0.6833881578947368}, {'decision': 2, 'accuracy': 0.7876712328767124, 'auc': 0.8099358974358974}]
______epoch 202 _____
train imitation 1.4742844104766846 reward 2.4974613189697266
val imitation 1.6928905248641968 reward 1.8477023839950562
val loss 3.540592908859253 3.2530345916

In [353]:
decision_model.get_embedding(decision_model.memory[1][300:600],position=1,use_saved_memory=True)

tensor([[1.4204, 0.7079, 0.5565,  ..., 0.0000, 0.0000, 0.0000],
        [0.2065, 0.4552, 0.7320,  ..., 0.0000, 0.0000, 0.0000],
        [0.3848, 0.1127, 0.0104,  ..., 0.0000, 0.0000, 0.6749],
        ...,
        [1.4807, 0.1248, 0.6973,  ..., 0.0000, 0.0000, 0.0000],
        [1.8474, 0.2117, 0.2452,  ..., 0.0000, 0.0000, 0.0000],
        [0.4463, 0.2753, 0.0000,  ..., 0.6557, 1.1129, 0.0000]],
       grad_fn=<ReluBackward0>)

In [342]:
decision_model.memory.shape

torch.Size([3, 2186, 82])

In [66]:
def gridsearch_decision_model(m1,m2,m3):
    model_arglist = [

        {
            'hidden_layers': [100,100],
            'attention_heads': [1,1],
        },
        {
            'hidden_layers': [50,50],
            'attention_heads': [1,1],
        },
        {
            'hidden_layers': [100,100],
            'attention_heads': [2,2],
        },
        {
            'hidden_layers': [300],
            'attention_heads': [3],
        },
        {
            'hidden_layers': [600],
            'attention_heads': [3],
        },
        {
            'hidden_layers': [300,300],
            'attention_heads': [3,3],
        },
        {
            'hidden_layers': [600,600],
            'attention_heads': [3,3],
        },
        {
            'hidden_layers': [1000],
            'attention_heads': [1],
        },
        {
            'hidden_layers': [1000],
            'attention_heads': [5],
        },
    ]
    best_loss = 100000000000
    best_metrics = {}
    best_args = {}
    best_model = None
    k = 0
    for margs in model_arglist:
        args = {k:v for k,v in margs.items()}
        for embed_size in [0,120,210]:
            #embed_size = 0 skips the firt layer that makes the sizes right
            if embed_size == 0 and args['attention_heads'][0] != 1:
                continue
            args['embed_size'] = embed_size
            for dropout in [.5,.9]:
                args['dropout'] = dropout
                for input_dropout in [.5]:
                    args['input_dropout'] = input_dropout
                    for shufflecol_chance in [.1,.5]:
                        args['shufflecol_chance'] = shufflecol_chance
                        model,m_metrics,m_loss = train_decision_model(m1,m2,m3,use_attention=True,verbose=False,**args)
                        print('done',k,m_loss)
                        print('curr best',best_loss)
                        k+=1
                        if m_loss < best_loss:
                            best_loss = m_loss
                            best_metrics  = m_metrics
                            best_model = model
                            best_args = args
                            print('_++++++++++New Best++++____')
                            print(best_loss)
                            print(best_metrics)
                            print(best_args)
                            print('___________')
                            print('++++++++')
                            print()
    print('_________')
    print('+++++++++++')
    print('best stuff',best_loss)
    print(best_metrics)
    print(best_args)
    return best_model

from Models import *
decision_model = gridsearch_decision_model(model1,model2,model3)
decision_model

/data/DigitalTwinVis/python/Preprocessing.py:182: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/data/DigitalTwinVis/python/Preprocessing.py:203: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  self.means = self.processed_df.mean(axis=0)
/data/DigitalTwinVis/python/Preprocessing.py:204: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or speci

++++++++++Final+++++++++++
best tensor(2.7922, grad_fn=<AddBackward0>)
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6634615384615385}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6461074561403509}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.728525641025641}]
done 0 tensor(2.7922, grad_fn=<AddBackward0>)
curr best 100000000000
_++++++++++New Best++++____
tensor(2.7922, grad_fn=<AddBackward0>)
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6634615384615385}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6461074561403509}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.728525641025641}]
{'hidden_layers': [100, 100], 'attention_heads': [1, 1], 'embed_size': 0, 'dropout': 0.5, 'input_dropout': 0.5, 'shufflecol_chance': 0.1}
___________
++++++++



/data/DigitalTwinVis/python/Preprocessing.py:182: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/data/DigitalTwinVis/python/Preprocessing.py:203: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  self.means = self.processed_df.mean(axis=0)
/data/DigitalTwinVis/python/Preprocessing.py:204: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or speci

++++++++++Final+++++++++++
best tensor(2.8300, grad_fn=<AddBackward0>)
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6725961538461539}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6639254385964912}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.7371794871794872}]
done 1 tensor(2.8300, grad_fn=<AddBackward0>)
curr best tensor(2.7922, grad_fn=<AddBackward0>)


/data/DigitalTwinVis/python/Preprocessing.py:182: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/data/DigitalTwinVis/python/Preprocessing.py:203: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  self.means = self.processed_df.mean(axis=0)
/data/DigitalTwinVis/python/Preprocessing.py:204: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or speci

...

/data/DigitalTwinVis/python/Preprocessing.py:182: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/data/DigitalTwinVis/python/Preprocessing.py:203: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  self.means = self.processed_df.mean(axis=0)
/data/DigitalTwinVis/python/Preprocessing.py:204: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or speci

++++++++++Final+++++++++++
best tensor(2.7552, grad_fn=<AddBackward0>)
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.68125}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.7044956140350876}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.7625}]
done 82 tensor(2.7552, grad_fn=<AddBackward0>)
curr best tensor(2.6872, grad_fn=<AddBackward0>)


/data/DigitalTwinVis/python/Preprocessing.py:182: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/data/DigitalTwinVis/python/Preprocessing.py:203: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  self.means = self.processed_df.mean(axis=0)
/data/DigitalTwinVis/python/Preprocessing.py:204: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or speci

++++++++++Final+++++++++++
best tensor(2.8200, grad_fn=<AddBackward0>)
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6600961538461538}, {'decision': 1, 'accuracy': 0.7671232876712328, 'auc': 0.6880482456140351}, {'decision': 2, 'accuracy': 0.8082191780821918, 'auc': 0.7612179487179487}]
done 83 tensor(2.8200, grad_fn=<AddBackward0>)
curr best tensor(2.6872, grad_fn=<AddBackward0>)
_________
+++++++++++
best stuff tensor(2.6872, grad_fn=<AddBackward0>)
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6817307692307693}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.7075109649122806}, {'decision': 2, 'accuracy': 0.815068493150685, 'auc': 0.7807692307692308}]
{'hidden_layers': [600], 'attention_heads': [3], 'embed_size': 210, 'dropout': 0.9, 'input_dropout': 0.5, 'shufflecol_chance': 0.5}


DecisionAttentionModel(
  (input_dropout): Dropout(p=0.5, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=210, out_features=600, bias=True)
  )
  (batchnorm): BatchNorm1d(600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.9, inplace=False)
  (sigmoid): Sigmoid()
  (softmax): LogSoftmax(dim=1)
  (final_layer): Linear(in_features=600, out_features=6, bias=True)
  (resize_layer): Linear(in_features=97, out_features=210, bias=True)
  (attentions): ModuleList(
    (0): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=210, out_features=210, bias=True)
    )
  )
  (norms): ModuleList(
    (0): LayerNorm((210,), eps=1e-05, elementwise_affine=True)
  )
  (activation): ReLU()
)

In [335]:
torch.save(decision_model,'../data/models/final_decision_model_' + decision_model.identifier + '.pt')
print('../data/models/final_decision_model_' + decision_model.identifier + '.pt')

../data/models/final_decision_model_statedecisions_input119_dims600,300_dropout0.5,0.9.pt


In [250]:
torch.save(model1,'../data/models/final_transition1_model_' + model1.identifier + '.pt')
torch.save(model2,'../data/models/final_transition2_model_' + model2.identifier + '.pt')
torch.save(model3,'../data/models/final_outcome_model_' + model3.identifier + '.pt')
print('../data/models/final_transition1_model_' + model1.identifier + '.pt')
print('../data/models/final_transition2_model_' + model2.identifier + '.pt')
print('../data/models/final_outcome_model_' + model3.identifier + '.pt')

../data/models/final_transition1_model_state1_input50_dims1000_dropout0.5,0.9.pt
../data/models/final_transition2_model_state2_input72_dims500,500_dropout0.5,0.9.pt
../data/models/final_outcome_model_state1_input70_dims500,500_dropout0.5,0.9.pt


In [349]:
torch.save(decision_model,'../resources/decision_model.pt')
torch.save(model1,'../resources/transition1_model.pt')
torch.save(model2,'../resources/transition2_model.pt')
torch.save(model3,'../resources/outcome_model.pt')

In [ ]:
xatt = []
for att,xxdf in zip(list(attributions),xdf):
    new = pd.DataFrame(att.cpu().detach().numpy(),columns=xxdf.columns,index=xxdf.index)
    xatt.append(new)
attributions = pd.concat(xatt,axis=1)
attributions

In [ ]:
attributions.sum().sort_values()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig = plt.subplots(1,1,figsize=(100,100))
sns.heatmap(data=attributions.T,ax=fig[1])

In [ ]:
def breakup_state_models(state_model):
    #for state 1 and 2
    models = {}
    models['pd'] = lambda x: state_model(x)[0]
    models['nd'] = lambda x: state_model(x)[1]
    models['chemo'] = lambda x: state_model(x)[2]
    for i,dlt in enumerate(Const.dlt1):
        models[dlt] = lambda x: state_model(x)[3][:,i]
    return models

def breakup_outcome_models(omodel):
    models = {}
    for i,name in enumerate(Const.outcomes):
        models[name] = lambda x: omodel(x)[:,i].reshape(-1,1)
    return models

def get_all_models(m1,m2,m3):
    state1_models = breakup_state_models(m1)
    state2_models = breakup_state_models(m2)
    state3_models = breakup_outcome_models(m3)
    all_models = {}
    for i,sm in enumerate([state1_models,state2_models,state3_models]):
        for ii,m in sm.items():
            all_models[ii +  '_state' + str(i+1)] = m
    return all_models

all_models = get_all_models(model,model2,model3)
all_models

In [ ]:

def get_ytrue(name,df):
    outcomes=None
    value = None
    if name == 'pd_state1':
        outcomes = df[Const.primary_disease_states]
    elif name == 'pd_state2':
        outcomes = df[Const.primary_disease_states2]
    elif name == 'nd_state1':
        outcomes = df[Const.nodal_disease_states]
    elif name == 'nd_state2':
        outcomes = df[Const.nodal_disease_states2]
    elif name == 'chemo_state1':
        outcomes = df[Const.modifications]
    elif name == 'chemo_state2':
        outcomes = df[Const.ccs]   
    if outcomes is not None:
        value = outcomes.idxmax(axis=1)
    if 'DLT' in name:
        newname = name.replace('_state', ' ').replace('1','').strip()
        value = df[newname]
    if name.replace('_state3','') in Const.outcomes:
        value = df[name.replace('_state3','')]
    if value is None:
        print(name,df.columns)
    return value

def check_impact_of_decisions(model_dict,data):
    results = []
    #todo: this is wrong fix it
    ids = []
    df = data.get_data()
    outcomedict = {step: pd.concat(data.get_intermediate_outcomes(step=step),axis=1) for step in [1,2,3]}
    for decision in Const.decisions:
        for name, model in model_dict.items():
            step = int(name[-1])
            subset0 = dataset.get_input_state(step=step,fixed={decision: 0})
            subset1 = dataset.get_input_state(step=step,fixed={decision: 1})
            outcomes = outcomedict[step]
            ids = subset0.index.values
            x0 = df_to_torch(subset0)
            x1 = df_to_torch(subset1)
            y0 = model(x0).detach().cpu().numpy()
            y1 = model(x1).detach().cpu().numpy()
            original = data.get_input_state(step=step)
            xx = df_to_torch(original)
            yy = model(xx).detach().cpu().numpy()
            ytrue = get_ytrue(name,outcomes)
            if "DLT" in name:
                y0 = y0.argmax(axis=1).reshape(-1,1)
                y1 = y1.argmax(axis=1).reshape(-1,1)
                yy = yy.argmax(axis=1).reshape(-1,1)
                change = y0 - y1
                decision_change = (y0 != y1).astype(int)
            elif y0.shape[1] == 1:
                change = y1 - y0
                decision_change = np.abs((y0 > .5).astype(int) - (y1 > .5).astype(int))
            else:
                index = np.unravel_index(np.argmax(yy, axis=1), yy.shape)
                change = (y0[index] - y1[index]).reshape(-1,1)
                decision_change =  (y0.argmax(axis=1).reshape(-1,1) != y1.argmax(axis=1).reshape(-1,1)).astype(int)
                yy = yy.argmax(axis=1).reshape(-1,1)
                y1 = y1.argmax(axis=1).reshape(-1,1)
                y0 = y0.argmax(axis=1).reshape(-1,1)
            outcome = name.replace('_state','')
            for ii,pid in enumerate(ids):
                oo = ytrue.loc[pid]
                onew = y0[ii][0]
                original_decision = df.loc[pid,decision]
                if original_decision > 0:
                    onew = y0[ii][0]
                oname = Const.name_dict.get(name)
                if oname is not None:
                    onew = oname[onew]
                entry = {'id': pid, 'decision': decision,'outcome': outcome,'original_choice': original_decision, 'original_result': oo, 'alt_result': onew, 'change': change[ii][0], 'decision_change': decision_change[ii][0]}
                results.append(entry)
    return pd.DataFrame(results)

test = check_impact_of_decisions(all_models,dataset)
test

In [ ]:
data.get_data()['SD Primary 2'].sum()

In [ ]:
(test[test.outcome == 'pd2'].original_result == 'SD Primary 2').sum()

In [ ]:
check_impact_of_decisions(all_models,dataset).to_csv('../data/decision_impacts.csv')